In [ ]:
#libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Define path to satellite image dataset
base_dir = '/kaggle/input/satellite-image-classification/data'

# Set configuration parameters
IMAGE_SIZE = (128, 128)  # Standard dimensions for neural network input
BATCH_SIZE = 32         # Number of images per training batch
EPOCHS = 20            # Number of complete passes through the dataset
DATA_PATH = base_dir   # Root directory containing image folders

# Initialize data generator with preprocessing options
datagen = ImageDataGenerator(
    rescale=1.0/255.0,     # Normalize pixel values to range [0,1]
    validation_split=0.2    # Reserve 20% of data for validation
)

# Create training data generator
train_data = datagen.flow_from_directory(
    DATA_PATH,              # Source directory
    target_size=IMAGE_SIZE, # Resize images
    batch_size=BATCH_SIZE,  # Images per batch
    class_mode='categorical', # Multi-class classification
    subset='training'       # Use training split
)

# Create validation data generator with same parameters
val_data = datagen.flow_from_directory(
    DATA_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'     # Use validation split
)

In [ ]:
# Define CNN architecture with increasing filters
model = Sequential([
    # First convolutional block - Initial feature extraction
    Conv2D(16, (3, 3), activation='relu', input_shape=(*IMAGE_SIZE, 3)),  # 16 filters, 3x3 kernel
    MaxPooling2D((2, 2)),  # Reduce spatial dimensions by half
    
    # Second convolutional block - Increased feature complexity
    Conv2D(32, (3, 3), activation='relu'),  # 32 filters for more feature maps
    MaxPooling2D((2, 2)),
    
    # Third convolutional block - Deep feature extraction
    Conv2D(64, (3, 3), activation='relu'),  # 64 filters for complex features
    MaxPooling2D((2, 2)),
    
    # Fourth convolutional block - Final feature extraction
    Conv2D(128, (3, 3), activation='relu'),  # 128 filters for detailed features
    MaxPooling2D((2, 2)),
    
    # Flatten layer to convert 2D feature maps to 1D vector
    Flatten(),
    
    # Dense layers for classification
    Dense(128, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Prevent overfitting by randomly dropping 50% of nodes
    Dense(train_data.num_classes, activation='softmax')  # Output layer - one node per class
])

# Configure model training parameters
model.compile(
    optimizer='adam',               # Adam optimizer for adaptive learning
    loss='categorical_crossentropy', # Loss function for multi-class classification
    metrics=['accuracy']            # Track accuracy during training
)

# Display model architecture summary
model.summary()

# Train model on dataset
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS                   # Train for specified number of epochs
)

In [ ]:
# Data augmentation and preprocessing configuration
datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values
    validation_split=0.2,      # 20% validation split
    rotation_range=30,         # Random rotation
    width_shift_range=0.2,     # Horizontal shift
    height_shift_range=0.2,    # Vertical shift
    horizontal_flip=True,      # Flip images horizontally
    zoom_range=0.2            # Random zoom
)

# Dataset configuration
IMG_SIZE = (64, 64)
data_dir = 'path'

# Generate training data with augmentation
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Generate validation data without augmentation
val_data = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Calculate class weights to handle imbalanced data
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights = dict(enumerate(class_weights))

# Load pre-trained VGG16 model
base_model = VGG16(
    weights='imagenet', 
    include_top=False, 
    input_shape=(*IMG_SIZE, 3)
)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

# Create final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train model
transfer = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15
)

# Evaluate model
val_loss, val_accuracy = model.evaluate(val_data)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Generate predictions and metrics
val_predictions = model.predict(val_data)
y_pred = val_predictions.argmax(axis=1)
y_true = val_data.classes

# Print classification metrics
print("\nClassification Report:")
print(classification_report(y_true, y_pred, 
      target_names=list(val_data.class_indices.keys())))

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)